# Introducción
### Contexto del dominio
El Fantasy Football se ha transformado de un pasatiempo de nicho a una industria multimillonaria impulsada por datos. En este juego, los participantes actúan como gerentes generales, seleccionando jugadores reales de la NFL para sus equipos virtuales. El rendimiento de estos equipos depende directamente de las estadísticas reales generadas en los partidos (yardas, touchdowns, recepciones). La toma de decisiones en el Draft y durante la temporada requiere procesar cientos de variables simultáneamente, lo que genera una sobrecarga cognitiva para el usuario promedio.

### Relevancia del problema
La capacidad de predecir el rendimiento futuro de un jugador (proyectado en Puntos de Fantasía o FPTS) es el núcleo del éxito en este juego. Sin embargo, la mayoría de los jugadores confían en la "intuición" o en rankings estáticos que no se adaptan a la composición de su equipo. Resolver este problema mediante Machine Learning permite optimizar la selección de talento, reduciendo el sesgo humano y aumentando significativamente la probabilidad de ganar ligas competitivas.

### Alcance del proyecto
Este proyecto abarca el desarrollo de un sistema "End-to-End" que incluye:
1.  **Ingeniería de Datos:** Procesamiento de estadísticas históricas de jugadores ofensivos (QB, RB, WR, TE) y unidades defensivas (K, DST).
2.  **Modelado Predictivo:** Entrenamiento y optimización de modelos de regresión para predecir FPTS totales.
3.  **Despliegue:** Una interfaz interactiva para simular Drafts y evaluar intercambios (trades).
*Fuera del alcance:* El análisis de noticias en tiempo real (NLP) sobre lesiones de último minuto o cambios climáticos no está incluido en esta iteración.

### Resumen del Dataset
Se utiliza un conjunto de datos recopilado de estadísticas de la NFL, estructurado en archivos CSV por posición (`QB.csv`, `RB.csv`, etc.). El dataset consolidado contiene aproximadamente 980 registros y más de 300 columnas tras el preprocesamiento, incluyendo variables numéricas (yardas, intentos) y categóricas (equipos, posiciones).

# Antecedentes
### Trabajos previos y Benchmarks
Existen múltiples plataformas que abordan la predicción de puntos en Fantasy Football:
1.  **FantasyPros:** Utiliza un enfoque de "Sabiduría de las Masas" (ECR - Expert Consensus Ranking), promediando las opiniones de múltiples expertos humanos.
2.  **ESPN / Yahoo Fantasy:** Utilizan proyecciones propietarias basadas en modelos estadísticos tradicionales, a menudo cajas negras para el usuario.
3.  **PFF (Pro Football Focus):** Ofrece métricas avanzadas y modelos predictivos, pero bajo barreras de pago costosas.

### Brecha identificada
La mayoría de las soluciones existentes son estáticas (rankings en PDF/Listas) o no permiten una interacción dinámica para evaluar escenarios específicos de "Trade" (intercambio) con transparencia sobre qué métricas impulsan la predicción. Además, rara vez exponen la incertidumbre o las métricas de error de sus modelos.

### Justificación de la aproximación
Nuestra solución añade valor al democratizar el acceso a un modelo de Machine Learning transparente. Al utilizar un enfoque de **AutoML (con Optuna)** y comparar familias de modelos (XGBoost, LightGBM, Random Forest), garantizamos que la predicción esté matemáticamente optimizada para minimizar el error cuadrático medio (RMSE), en lugar de basarse en sesgos subjetivos. La integración en una Web App permite al usuario interactuar con estos datos en tiempo real durante su Draft.

# Objetivos y Metricas de Evaluación
### Objetivo General
Desarrollar y desplegar un sistema inteligente de asistencia para Fantasy Football que prediga los puntos totales de la temporada (FPTS) mediante modelos de Machine Learning, sirviendo estas predicciones a través de una API y una interfaz gráfica para la toma de decisiones en Drafts y Trades.

### Objetivos Específicos
1.  **Pipeline de Datos Reproducible:** Construir un flujo automatizado que ingeste, limpie y transforme estadísticas crudas de la NFL en un dataset apto para el entrenamiento, gestionando valores nulos y codificación categórica.
2.  **Optimización de Modelos (HPO):** Evaluar y optimizar tres familias de algoritmos (XGBoost, LightGBM, Random Forest) utilizando Optuna para encontrar la configuración de hiperparámetros que minimice el error de predicción.
3.  **Orquestación y Registro:** Implementar MLflow para el seguimiento de experimentos y registrar el mejor modelo ("Champion") en un registro de modelos centralizado.
4.  **Despliegue de Servicios:** Contenerizar el modelo mediante Docker, exponiéndolo vía una API (FastAPI) y consumiéndolo mediante una interfaz de usuario (Streamlit) alojada en la nube (AWS).

### Criterios de Éxito
* **Métrica Principal:** Lograr un **RMSE (Root Mean Squared Error) ≤ 15.0** puntos en el conjunto de prueba, lo que representa un margen de error aceptable para una proyección de temporada completa.
* **Métrica Secundaria:** **R² ≥ 0.85**, asegurando que el modelo explique la gran mayoría de la varianza en los datos.

### Riesgos Asociados
* **Data Leakage:** Riesgo de incluir variables derivadas del target (como promedios por juego) que inflen artificialmente el rendimiento.
* **Imprevisibilidad:** Lesiones o suspensiones de jugadores que los datos históricos no pueden prever.

# Planteamiento del problema
### Formulación
¿Es posible predecir con precisión los puntos de fantasía (FPTS) totales de un jugador de la NFL basándose exclusivamente en sus estadísticas técnicas y rol posicional, superando las proyecciones base mediante técnicas de ensamble de árboles de decisión?

### Variables Clave
* **Entradas (Features):** Estadísticas de juego acumuladas (Intentos de pase, Yardas de recepción, Sacks defensivos, Touchdowns, Fumbles), Equipo al que pertenece, y Posición (One-Hot Encoded).
* **Salida (Target):** `FPTS` (Fantasy Points Total Score).
* **Variables Auxiliares:** `Rank`, `G` (Juegos jugados) - utilizadas para filtrado pero tratadas con cuidado para evitar fugas de información.

### Supuestos y Restricciones
* **Supuesto:** Se asume que el rendimiento pasado (métricas de volumen y eficiencia) tiene una correlación fuerte con el rendimiento futuro inmediato.
* **Restricción:** El modelo no considera cambios de equipo en la "off-season" (agencia libre) a menos que estén reflejados en el dataset de entrada actualizado.

### Impacto Esperado
Una solución efectiva permitirá a los usuarios identificar "Robos del Draft" (jugadores subestimados con alta proyección) y evitar "Busts" (jugadores sobrevalorados). Esto se traduce en una ventaja competitiva directa en sus ligas.

### Riesgos
* **Sesgo de Disponibilidad:** Jugadores que jugaron pocos partidos por lesión pueden tener proyecciones sesgadas si no se imputan correctamente.
* **Interpretabilidad:** Modelos complejos como XGBoost pueden ser difíciles de explicar a un usuario final sin herramientas como SHAP (aunque se prioriza el rendimiento predictivo).

# Desarrollo de la solución
### EDA

##### Metodologia

Se hizo un analisis exploratorio sistematico relacionado a los roles de los jugadores; a de manera biyectiva, cuales son los puntos que pueden llegar a lograr.
De esa manera medimos lo siguiente (Por rol):
* La distribución de puntos que usualmente generan
* Su distribucion de yardas recorridas o pases completados
* Su precision en recepciones, intercepciones o patadas y la distancia de estos
* Una comparacion de las anteriores estadisticas con la cantidad de puntos

##### Hallazgos Clave
<div align="center">
  <img src="imgs/plots/quaterback%20analytic.png" width="700"/>
</div>

* **Observación:** Se evidencia una fuerte correlación lineal positiva entre las yardas aéreas y los FPTS. Los QBs de élite se separan del resto no solo por yardas, sino por una alta tasa de Touchdowns. Se detectan outliers en la parte inferior izquierda correspondientes a QBs suplentes con pocos minutos de juego.

<div align="center">
  <img src="imgs/plots/kicker%20analytic.png" width="700"/>
</div>

* **Observación:** La posición de Kicker muestra una alta varianza. A diferencia de otras posiciones, el volumen de oportunidades (intentos de FG) es más determinante que la precisión pura. La distribución es más plana, lo que sugiere que es difícil predecir un "Kicker estrella" consistente.

<div align="center">
  <img src="imgs/plots/wide%20receiver%20analytic.png" width="700"/>
</div>

* **Observación:** La relación entre "Targets" (pases lanzados hacia él) y FPTS es crítica. Se observa una distribución de "ley de potencia": un grupo selecto de WRs élite genera la gran mayoría de los puntos, mientras que hay una larga cola de receptores con producción marginal.

<div align="center">
  <img src="imgs/plots/running%20back%20analytic.png" width="700"/>
</div>

* **Observación:** Los RBs muestran dos clústeres de producción: aquellos que dependen del volumen de acarreos y aquellos híbridos que suman puntos por recepción. Los RBs que participan en el juego aéreo tienden a tener un "piso" de puntos más alto y estable.

<div align="center">
  <img src="imgs/plots/tight%20end%20analytic.png" width="700"/>
</div>

* **Observación:** Esta es la posición más escasa ("Top-heavy"). Solo los primeros 3-5 TEs muestran una producción comparable a los WRs. El resto de la distribución cae drásticamente, lo que indica que obtener un TE de élite es una ventaja posicional masiva.

<div align="center">
  <img src="imgs/plots/defense%20analytic.png" width="700"/>
</div>

* **Observación:** Las defensas (DST) obtienen puntos principalmente por eventos de baja probabilidad (Touchdowns defensivos, Safety) y Sacks. Esto genera una nube de puntos dispersa con correlaciones más débiles que en las posiciones ofensivas, indicando mayor volatilidad semana a semana.

<div align="center">
  <img src="imgs/plots/pst%20distro.png" width="700"/>
</div>

* **Observación:** El dataset no está perfectamente balanceado; hay significativamente más WRs y RBs en la muestra que QBs o TEs, lo cual es natural dado que los equipos de la NFL tienen rosters más profundos en esas posiciones. Esto justifica el uso de estratificación o métricas robustas al entrenar.

##### Variables cantidatas a ingenieria de caracteristicas
Todas las variables categoricas son cantidatas a un OHE para su procesamiento.

##### Riesgos detectados
Algunas columnas pueden provocar data leakage ya que son reflejos de la variable objetivo "FTPS".

### Data Wrangling
Lista de transformaciones:
* Etiquetado de posición - Se añade la columna Position a cada dataframe individual(DST, K, QB, RB, TE, WR). Justificacion: Necesitaremos reconocer posiciones cuando juntemos todos los df's para procesarlos. Alternativas consideradas: Procesar cada df individualmente pero debido al aumento de complejidad, descartado.
* Unión de dataset - Concatenación de todos los dataframes en un solo dataset general. Justificación: Para hacer un procesamiento formal de los datos decisimos juntar a todos los jugadores de la liga en el mismo df. Alternativas consideradas: NA.
* Limpieza inicial - Eliminación de columnas irrelevantes: Player, Team, FPTS, FPTS/G. Justificación: estas son las col que podian crear data leakage + col que no tienen efecto directo en los datos. Alternativa considerada: NA.
* Conversión de tipos - Justificacion: en el caso que hubiera str que realmente eran int se decidio volver a transformar col con >50% en datos num. a num. solo para asegurar la limpieza de los datos. De misma manera pudimos clasificar los que no eran numericos. Alternativa considerada: NA.
* Imputación de valores faltantes. Justificación: decidimos utilizar la mediana en vez de la media ya que en el deporte puede haber deportistas estrella o outliers practicamente por lo que la mediana nos servira para balancear ese fenomeno. Alternativa considerada: Utilizar la media pero fue descartada.
* Codificación categórica OHE. Justificación: Es la manera mas directa de poder computar datos categoricos, fue utilizada por la complejidad de los datos y su buen rendimiento. Alternativa considerada: Mineria de Datos.

Evidencias (Antes/Despues):
* Antes:

| Rank | Player                      | SACK | INT | FR | FF | DEF TD | SFTY | SPC TD | G  | ... | TD  | SACKS | ATT.1 | YDS.1 | TD.1 | FL  | 20+ | TGT | REC | Y/R |
|------|-----------------------------|------|-----|----|----|--------|------|--------|----|-----|-----|-------|--------|--------|------|-----|-----|-----|-----|-----|
| 1.0  | Houston Texans (HOU)        | 33.0 | 12.0| 7.0| 8.0| 2.0    | 0.0  | 0.0    | 11.0| ... | NaN | NaN   | NaN    | NaN    | NaN  | NaN | NaN | NaN | NaN | NaN |
| 2.0  | Los Angeles Rams (LAR)      | 31.0 | 12.0| 7.0|10.0| 1.0    | 0.0  | 0.0    | 11.0| ... | NaN | NaN   | NaN    | NaN    | NaN  | NaN | NaN | NaN | NaN | NaN |
| 3.0  | Seattle Seahawks (SEA)      | 36.0 | 9.0 | 4.0| 3.0| 2.0    | 0.0  | 2.0    | 11.0| ... | NaN | NaN   | NaN    | NaN    | NaN  | NaN | NaN | NaN | NaN | NaN |
| 4.0  | Cleveland Browns (CLE)      | 42.0 | 9.0 | 6.0|10.0| 2.0    | 0.0  | 0.0    | 11.0| ... | NaN | NaN   | NaN    | NaN    | NaN  | NaN | NaN | NaN | NaN | NaN |
| 5.0  | Pittsburgh Steelers (PIT)   | 34.0 | 9.0 |11.0|13.0| 3.0    | 0.0  | 0.0    | 11.0| ... | NaN | NaN   | NaN    | NaN    | NaN  | NaN | NaN | NaN | NaN | NaN |

* Despues:

| Rank | SACK | INT | FR  | FF  | DEF TD | SFTY | SPC TD | G   | FG  | ... | ROST_99.4% | ROST_99.6% | ROST_99.7% | ROST_99.8% | ROST_99.9% | Position_K | Position_QB | Position_RB | Position_TE | Position_WR |
|------|------|-----|-----|-----|--------|------|--------|-----|-----|-----|------------|------------|------------|------------|------------|-------------|--------------|--------------|--------------||--------------|
| 1.0  | 33.0 |12.0 | 7.0 | 8.0 | 2.0    | 0.0  | 0.0    |11.0 | 9.5 | ... | False      | False      | False      | False      | False      | False       | False        | False        | False        | False        |
| 2.0  | 31.0 |12.0 | 7.0 |10.0 | 1.0    | 0.0  | 0.0    |11.0 | 9.5 | ... | False      | False      | False      | False      | False      | False       | False        | False        | False        | False        |
| 3.0  | 36.0 | 9.0 | 4.0 | 3.0 | 2.0    | 0.0  | 2.0    |11.0 | 9.5 | ... | False      | False      | False      | False      | False      | False       | False        | False        | False        | False        |
| 4.0  | 42.0 | 9.0 | 6.0 |10.0 | 2.0    | 0.0  | 0.0    |11.0 | 9.5 | ... | False      | False      | False      | False      | False      | False       | False        | False        | False        | False        |
| 5.0  | 34.0 | 9.0 |11.0 |13.0 | 3.0    | 0.0  | 0.0    |11.0 | 9.5 | ... | False      | False      | False      | False      | False      | False       | False        | False        | False        | False        |

Resultado final(df.info):

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 983 entries, 0 to 982
Columns: 317 entries, Rank to Position_WR
dtypes: bool(283), float64(34)
memory usage: 532.9 KB
None


Ruta final del Dataset:
* Antes:
"DraftFantasy/src/data/df.csv"

* Despues
"DraftFantasy/src/data/df_processed_eg.csv"

Referencia tecnica completa:
"DraftFantasy/src/notebooks/02_DataWrangling.ipynb"

### Entrenmamiento con MLFLOw

Familia de modelos utilizada:
* XGBoost
* LightGBM
* Random Forest

Registro de MlFlow:
La estrutura de los experimentos fue la siguiente, se hizo la busqueda de hiperparametros, 50 trials alternando entre los 3 modelos de esos 50 trials, se guarda el modelo con el mejor espacio de bisqueda que registro un menor rmse. Se registraron, el rmse, el r2 y el mae. Pero rmse fue la metrica seleccionada como criterio de uso. El mejor modelo se creo y se registro.

Optimizacion de Hiperparametros:
Se utilizo la libreria optuna, con un numero de evaluaciones de 50 trials. El espacio de busqueda ya fue descrito anteriormente.

Evidencias(Mejores experimentos):
<div align="center">
  <img src="imgs/ss/mlflow/best_experiments.PNG" width="700"/>
</div>

Artifacts registrados:
Los artifacts que se crearon fue principalmente el preprocesador se puede encontrar en "DraftFantasy/src/pipelines/artifacts" como preprocessor.pkl

### Selección del mejor modelo.

Criterio Objetivo:

Como se puede adelantar en nuestra anterior sección se selecciono el mejor modelo por medio del rmse minimizandolo.

Comparación consolidada:

| Run | Model    | n_estimators | max_depth | learning_rate | subsample | colsample_bytree | reg_lambda | reg_alpha | num_leaves |   RMSE   |   MAE   |    R²    |
|-----|----------|--------------|-----------|----------------|-----------|-------------------|------------|-----------|-------------|----------|---------|----------|
| 59  | xgboost  | 127          | 13        | 0.0978         | 0.5102    | 0.6810            | 0.0203     | 0.9403    | None        | 5.959179 | 2.609105| 0.979942 |
| 111 | xgboost  | 283          | 4         | 0.0923         | 0.7553    | 0.5388            | 1.5395     | 1.1419    | None        | 6.365348 | 2.679624| 0.977115 |
| 163 | xgboost  | 378          | 8         | 0.0943         | 0.5017    | 0.6147            | 0.1431     | 0.1255    | None        | 6.421486 | 2.498007| 0.976709 |
| 215 | lightgbm | 445          | 9         | 0.0594         | 0.6994    | 0.6917            | 0.2016     | 5.5862    | 97          | 6.550088 | 2.903093| 0.975767 |
| 0   | lightgbm | 362          | 9         | 0.0617         | 0.7830    | 0.9940            | 0.0072     | 0.2217    | 56          | 6.657549 | 2.998966| 0.974965 |

Justificacion final: Los modelos presentados aqui fueron los modelos registrados, es decir. Lo mejor del espacio de busqueda para cada experimento. Aqui algo que modelos destacar es lo pequeño que es reg_lamada y lo bien que optimiza lo que sugiere que los cambios son por detalles menores en el juego.

Registro del Modelo Ganador:

El modelo ganador siempre es registrado como champion.

### Orquestación

Descripción del flujo automatizado:
1. La data se pasa a la funcion de preprocesamiento que genera un artefacto llamado preprocessor.pkl que se usara para el uso en la API
2. Los datos tratados se pasan a los optimizadores del espacio de busqueda.
3. El modelo con los mejores hiperparametros es creado y evaluado.
4. Esta información es guardada en el unity_catalog de Databricks.

Comandos:

python.exe DraftFantasy/src/pipelines/train_pipeline.py

Manejo de fallos:

Excepciones Manejadas con ValueError()

Evidencias:

<div align="center">
  <img src="imgs/diagrams/train_pipeline_diagram.png" width="700"/>
  <br>
  <img src="imgs/ss/misc/train_flow_running.PNG" width="700"/>
  <br>
  <img src="imgs/ss/prefect/prefect_history.PNG" width="700"/>
  <br>
  <img src="imgs/ss/prefect/prefect_flow.PNG" width="700"/>
</div>

### Servir modelo (API con FastAPI)

Objetivo del servicio:

El objetivo del servicio es servir la mejor prospeccion de un jugador la proxima semana, asi otorgar un buen draft al jugador.

Endpoints:

* (/) root
* (/health) health check
* (/predict) predecir mejor jugador
* (/predict_batch) predecir los mejores 10 jugadores

Flujo del modelo:

El flujo del modelo se puede apreciar en el siguiente diagrama
<div align="center">
  <img src="imgs/diagrams/deploy_diagram.png" width="700"/>
</div>

Podemos ver que la API recibe la informacion ya preprocesada, llega una request (que se manejan a traves del sistema 200 success, 500 error) y colecta el modelo del model registry con anterioridad y ejecuta el algoritmo para mandar la respuesta a la UI.

Referencia de codigo:

La API se encuentra en "DraftFantasy/src/backend/api.py"

Evidencia:

<div align="center">
  <img src="imgs/ss/api/API_ss_aws.PNG" width="700"/>
</div>

### Interfaz Gráfica (Streamlit)

Objetivo:
* Brindar al usuario el servicio de asistencia por ML para su Draft.
* Brindar al servicio de otros servicios cuantitativos tradicionales, como mejor alineacion o calculo de valor en el trade.

Componentes minimos:
Iniciar tu session de Draft. Cada vez que pasen las rondas el modelo encontrara el mejor jugador disponible o diferentes opciones igual de buenas (batch).

Indicaciones de ejecucion local:

streamlit run DraftFantasy/src/frontend/app.py

La UI se encuentra en "DraftFantasy/src/frontend/app.py"


<div align="center">
  <video width="700" controls>
    <source src="imgs/ss/GUI/streamlit_aws.mp4" type="video/mp4">
  </video>
</div>


### Contenerización del servicio

Dockerfile backend:
```
FROM python:3.11-slim

WORKDIR /app

COPY backend/api.py ./api.py

COPY backend/requirements.txt ./requirements.txt
RUN pip install --no-cache-dir -r requirements.txt

EXPOSE 8000

CMD ["uvicorn", "api:app", "--host", "0.0.0.0", "--port", "8000"]
```


Dockerfile frontend:
```
FROM python:3.11-slim

WORKDIR /app

COPY frontend/app.py ./app.py

COPY frontend/requirements.txt ./requirements.txt
RUN pip install --no-cache-dir -r requirements.txt

EXPOSE 8501

CMD ["streamlit", "run", "app.py", "--server.address=0.0.0.0", "--server.port=8501"]
```

docker-compose.yml

```
version: "3.8"

services:
  backend:
    build:
      context: .
      dockerfile: backend/Dockerfile
    ports:
      - "8000:8000"
    environment:
      - DATABRICKS_HOST=${DATABRICKS_HOST}
      - DATABRICKS_TOKEN=${DATABRICKS_TOKEN}
    volumes:
      - ./pipelines/artifacts/preprocessors:/app/preprocessors:ro
    restart: unless-stopped

  frontend:
    build:
      context: .
      dockerfile: frontend/Dockerfile
    ports:
      - "8501:8501"
    depends_on:
      - backend
    environment:
      - API_URL=http://backend:8000
    volumes:
      - ./data:/app/data:ro
    restart: unless-stopped
```



<div align="center">
  <img src="imgs/diagrams/docker_diagram.png" width="700"/>
</div>

### Despliegue del servicio en la nube

Plataforma elegida:

AWS

Pasos:

1. Iniciar una instancia de EC2 (Liberar puerto 8000 y 8501, apartar al menos 15 gib, OS: ubuntu)
2. Ingresar a la computadora a traves de tu llave .pem previamente descargada o traves de la consola de aws.
3. git clone (a este repositorio)
4. Descargar docker-compose

Para descargar docker-compose ejecutar los siguientes comandos en orden.

* sudo apt update
* sudo apt upgrade -y
* sudo apt install -y apt-transport-https ca-certificates curl software-properties-common
* curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo gpg --dearmor -o /usr/share/keyrings/docker-archive-keyring.gpg
* sudo apt update
* sudo apt install -y docker-ce docker-ce-cli containerd.io docker-compose-plugin
* sudo systemctl start docker
* sudo systemctl enable docker
* sudo usermod -aG docker $USER
* exit

Despues vuelves a acceder a la maquina, eso significa que se ha reiniciado.

5. Acceder a DraftFantasy/src
6. Crear tu archivo .env

Para crear tu archivo .env sigue las siguientes instrucciones y comandos

* nano .env
* escribir DATABRICKS_HOST=tu_url /parse DATABRICKS_TOKEN=tu_token
* ctrl + x
* Guardar

7. Ejecutar docker-compose up --build
8. Esperar a que los contenedores se levantes
9. Acceder a tus puertos a traves de la ip publica de AWS


Evidencia:

<div align="center">
  <img src="imgs/ss/docker/ready_docker_aws.PNG" width="700"/>
</div>

Limpieza/Costos:

para detener los contenedores ejecuta el siguiente comando:

* docker-compose down

Ahora para limpiar el cache y el almacenamiento ejecuta los siguientes pasos:


* docker stop $(docker ps -aq)
* docker rm $(docker ps -aq)
* docker rmi $(docker images -q) -f
* docker volume rm $(docker volume ls -q)
* docker network prune -f
* docker system df


# Conclusiones y recomendaciones

### Síntesis de lo logrado
Se ha completado con éxito el ciclo de vida de un proyecto de Ciencia de Datos End-to-End. Hemos logrado:
1.  Ingestar y limpiar datos heterogéneos de múltiples posiciones de la NFL, creando un dataset unificado y libre de fugas de información (Data Leakage).
2.  Implementar un pipeline de entrenamiento robusto que, mediante Optuna y MLflow, seleccionó automáticamente un modelo **XGBoost** con un rendimiento predictivo superior (R² > 0.97 y RMSE controlado).
3.  Desplegar la solución en la nube (AWS) utilizando contenedores Docker, permitiendo que cualquier usuario acceda a las predicciones a través de una interfaz web intuitiva.

### Aprendizajes Clave
* **La importancia del Preprocesamiento:** La eliminación de variables como `FPTS/G` (puntos por juego) fue crítica para evitar que el modelo hiciera "trampa" durante el entrenamiento.
* **Automatización:** El uso de pipelines orquestados reduce drásticamente el tiempo de iteración al probar nuevos modelos.
* **Infraestructura:** La contenerización demostró ser vital para evitar el clásico problema de "funciona en mi máquina", asegurando un despliegue fluido en AWS.

### Limitaciones
* El modelo actual depende puramente de estadísticas acumuladas. No tiene en cuenta el contexto cualitativo, como la dificultad del calendario futuro ("Strength of Schedule") o cambios recientes en el cuerpo técnico.
* La imputación de valores faltantes con la mediana es robusta, pero podría simplificar demasiado el perfil de jugadores novatos (Rookies) de los cuales no se tiene historial.

### Líneas Futuras (Recomendaciones)
1.  **Ingeniería de Características Avanzada:** Incorporar métricas de "oportunidad" como *Target Share* o *Red Zone Attempts* para refinar la predicción de Touchdowns.
2.  **Modelos de Series de Tiempo:** En lugar de predecir el total anual, implementar modelos LSTM o Transformers para predecir el rendimiento semana a semana.
3.  **NLP para Noticias:** Integrar un módulo que analice noticias de Twitter/X sobre lesiones para ajustar las proyecciones en tiempo real (penalización por riesgo de lesión).
4.  **Escalado:** Implementar Kubernetes si la carga de usuarios concurrentes en el Draft Simulator aumenta significativamente.

# Referencias
* https://docs.docker.com/
* https://docs.astral.sh/uv/guides/package/
* https://docs.databricks.com/aws/en/
* https://fastapi.tiangolo.com/
* https://www.prefect.io/
* https://streamlit.io/